In [ ]:
import pandas as pd
import requests

# --- 1. Leer el archivo limpio (usa una de las tres opciones arriba) ---
file_path = r"C:\Deysi\ironHack\ironHack\projects\Educacion\data\Datos_de_asistencia_escolar.csv"
df = pd.read_csv(file_path)

# --- 2. Convertir la columna de asistencia escolar a numérica ---
df['school_attendance'] = pd.to_numeric(df['school_attendance'], errors='coerce')

# --- 3. Eliminar valores nulos o no válidos ---
df = df.dropna(subset=['school_attendance'])

# --- 4. Ordenar por menor asistencia escolar (más pobreza educativa) ---
df_sorted = df.sort_values(by='school_attendance', ascending=True)

# --- 5. Mostrar los 30 países más críticos ---
top30 = df_sorted.head(30)

print("🏫 Los 30 países con menor asistencia escolar (mayor pobreza educativa):\n")
print(top30[['country_name', 'country_code', 'school_attendance', 'year']].to_string(index=False))

# --- 1. Definir URL de la API ---
# Indicador: "SI.POV.DDAY" → población que vive con menos de $2.15/día (ajustado por paridad de poder adquisitivo)
# Puedes ajustar el rango de años o los países
url = "https://api.worldbank.org/v2/country/all/indicator/SI.POV.DDAY?format=json&date=2010:2022&per_page=20000"

# --- 2. Descargar los datos ---
response = requests.get(url)
data = response.json()[1]  # el índice [1] contiene los registros

# --- 3. Convertir a DataFrame ---
wb_df = pd.DataFrame(data)

# --- 4. Extraer y limpiar columnas relevantes ---
wb_df = wb_df[['country', 'date', 'value']]
wb_df['country_name'] = wb_df['country'].apply(lambda x: x['value'])
wb_df['country_code'] = wb_df['country'].apply(lambda x: x['id'])
wb_df.rename(columns={'value': 'poverty_rate', 'date': 'year'}, inplace=True)
wb_df['year'] = wb_df['year'].astype(int)

# --- 5. Mantener columnas útiles ---
wb_df = wb_df[['country_code', 'country_name', 'year', 'poverty_rate']]

# --- 6. Limpiar valores nulos ---
wb_df.dropna(subset=['poverty_rate'], inplace=True)

# --- 7. Mostrar algunos resultados ---
print("✅ Datos del Banco Mundial cargados correctamente:")
print(wb_df.head(10))

print("\nNúmero de países registrados:", wb_df['country_name'].nunique())


print("Años en UNICEF:", sorted(df['year'].unique()))
print("Años en Banco Mundial:", sorted(wb_df['year'].unique()))

print("UNICEF country codes:", sorted(df['country_code'].unique()))
print("World Bank country codes:", sorted(wb_df['country_code'].unique()))
common = set(df['country_code']).intersection(set(wb_df['country_code']))
print("Códigos comunes:", common)
print("Número:", len(common))

